In [1]:
import tensorflow as tf
import pickle
import sys
import matplotlib.pyplot as plt1 
import numpy as np
from IPython.display import Audio, display
import time
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate, Conv1D,Conv2D, Flatten, Reshape, Embedding, GRU, SpatialDropout1D, LSTM, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from itertools import permutations
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from scipy.stats import trim_mean

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
full = pd.read_csv("../data/drug_class_identification/all3.csv")
full = full.dropna()
full['atc'] = full['atc'].apply(lambda x : x[0])
full.head()

,smiles,name,id,inchi_key,atc
75,NC12CC3CC(CC(C3)C1)C2,amantadine,BRD-K70330367,DKNWSYNQZKUICI-UHFFFAOYSA-N,N
291,CS(=O)(=O)OCCCCOS(C)(=O)=O,busulfan,BRD-K23204545,COVZYZSDYWQREU-UHFFFAOYSA-N,L
322,CCCC(C)(COC(N)=O)COC(=O)NC(C)C,carisoprodol,BRD-A99939097,OFZCIYFFPZCNJE-UHFFFAOYSA-N,M
491,CCN(CC)C(=O)N1CCN(C)CC1,diethylcarbamazine,BRD-K45542189,RCKMWOKWVGPNJF-UHFFFAOYSA-N,P
510,CCN(CC)C(=S)SSC(=S)N(CC)CC,disulfiram,BRD-K32744045,AUZONCFQVSMFAP-UHFFFAOYSA-N,P


In [3]:
# Define our own plot function
def scatter(x, y, subtitle=None):
    le = LabelEncoder()
    labels = le.fit_transform(y)

    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 3))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[labels.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(3):
        # Position of each label.
        xtext, ytext = trim_mean(x[labels == i, :], axis=0, proportiontocut=0.2)
        letter = le.inverse_transform([i])[0]
        txt = ax.text(xtext, ytext, str(letter), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
        
    if subtitle != None:
        plt.suptitle(subtitle)
        
    plt.savefig(subtitle)

In [4]:
snn = pd.read_csv('EMB_snn_4_25_16_full-75')
snn['id'] = snn['pert_id']
snn1 = snn.groupby(['id']).mean()
snn1.shape

(2170, 17)

In [5]:
snn.head()

,Unnamed: 0,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,pert_id,id
0,0,-0.029784,0.310603,0.277136,-0.431018,-0.357303,0.261168,-0.271236,0.341609,-0.026670,-0.366111,0.026679,-0.160398,0.023314,0.094788,0.012563,0.288174,BRD-A00147595,BRD-A00147595
1,1,-0.169498,0.467458,0.297595,-0.340158,-0.123368,0.289200,-0.290569,0.131418,0.100359,-0.009926,-0.165926,-0.148920,0.201667,-0.093444,-0.073614,0.482936,BRD-A00147595,BRD-A00147595
2,2,-0.119119,0.449742,0.225728,-0.502185,-0.240132,0.243425,-0.181789,0.013748,0.161168,-0.159822,-0.139383,0.047707,0.079915,-0.163043,-0.138084,0.452755,BRD-A00147595,BRD-A00147595
3,3,0.286788,0.037963,0.203989,-0.473443,0.028442,0.181074,-0.079908,0.321214,0.089437,-0.213721,0.204931,-0.200505,0.070095,0.045062,-0.191935,0.572480,BRD-A00147595,BRD-A00147595
4,4,0.043722,0.415972,0.326342,-0.278743,-0.430512,-0.152955,-0.095241,0.413184,-0.117863,-0.165499,0.145251,-0.290483,0.064071,0.273093,0.162439,0.022388,BRD-A00147595,BRD-A00147595


In [6]:
result_snn = pd.merge(snn, full, how="inner").drop(['id','smiles','name','inchi_key','pert_id','Unnamed: 0'],axis=1).dropna()
result_snn = result_snn[result_snn.atc.isin(['C','L','N'])]

In [7]:
y_snn = result_snn['atc']
X_snn = result_snn.drop(['atc'],axis=1)
X_snn = np.asarray(X_snn)
result_snn.head()

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,atc
84,0.511453,-0.057744,0.340139,-0.015155,-0.317840,0.194005,-0.037603,0.076650,0.126349,-0.204697,-0.163801,-0.094300,0.273752,-0.383546,-0.200301,0.342713,N
85,0.229944,-0.020185,0.443398,-0.518480,-0.075177,-0.075593,-0.211682,0.183089,0.246457,-0.213850,0.084293,-0.043345,0.243932,0.088033,-0.061777,0.452849,N
86,0.406272,0.178916,0.228196,-0.407354,-0.105116,-0.066051,-0.042662,0.194615,-0.137083,-0.137357,0.093827,-0.001609,0.320882,-0.158149,-0.219738,0.554155,N
87,0.128144,-0.023080,0.068159,-0.244413,-0.179207,0.012507,-0.069012,0.460512,0.269545,-0.259429,-0.080124,-0.226373,0.337361,-0.243033,-0.068072,0.542609,N
88,-0.021558,0.286940,0.314538,-0.348253,-0.328626,0.098346,-0.077093,0.218763,0.190059,-0.225834,-0.017855,-0.050591,0.522465,0.003779,-0.071582,0.396750,N


In [8]:
X_snn.shape
le = LabelEncoder()
labels = le.fit_transform(y_snn)

In [9]:
x_train_snn, x_test_snn, y_train_snn, y_test_snn = train_test_split(X_snn,labels)

x_train_flat_snn = x_train_snn.reshape(-1,16)
x_test_flat_snn = x_test_snn.reshape(-1,16)

tsne = TSNE()
train_tsne_embeds_snn = tsne.fit_transform(x_train_flat_snn[:1000])
scatter(train_tsne_embeds_snn, y_train_snn[:1000], "Samples from Training Data LINCS (DenseNet) Before")

eval_tsne_embeds_snn = tsne.fit_transform(x_test_flat_snn[:1000])
scatter(eval_tsne_embeds_snn, y_test_snn[:1000], "Samples from Validation Data LINCS (DenseNet) Before")

/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)


In [10]:
def baseNetwork(op):    
        model = Sequential()
        model.add(Dense(50, activation='softmax', input_dim=16))
        model.add(Dropout(0.5))
        model.add(Dense(20, activation='softmax'))
        model.add(Dropout(0.5))
        model.add(Dense(10, activation='softmax'))
        model.add(Dropout(0.5))
        model.add(Dense(op, activation='softmax'))
        return model

In [11]:
model = baseNetwork(3)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                850       
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 33        
Total para

In [12]:
model.fit(x_train_snn,y_train_snn, epochs=500)

Epoch 1/500
18819/18819 [==============================] - 1s 73us/step - loss: 1.0451 - acc: 0.4897
Epoch 2/500
18819/18819 [==============================] - 1s 48us/step - loss: 1.0326 - acc: 0.5125
Epoch 3/500
18819/18819 [==============================] - 1s 49us/step - loss: 1.0206 - acc: 0.5125
Epoch 4/500
18819/18819 [==============================] - 1s 50us/step - loss: 0.9720 - acc: 0.5212
Epoch 5/500
18819/18819 [==============================] - 1s 55us/step - loss: 0.9272 - acc: 0.5378
Epoch 6/500
18819/18819 [==============================] - 1s 54us/step - loss: 0.9028 - acc: 0.5441
Epoch 7/500
18819/18819 [==============================] - 1s 63us/step - loss: 0.8861 - acc: 0.5529
Epoch 8/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.8783 - acc: 0.5533
Epoch 9/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.8655 - acc: 0.5605
Epoch 10/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.8625 - a

18819/18819 [==============================] - 1s 59us/step - loss: 0.7482 - acc: 0.6142
Epoch 82/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7450 - acc: 0.6189
Epoch 83/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7425 - acc: 0.6196
Epoch 84/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7458 - acc: 0.6169
Epoch 85/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7450 - acc: 0.6171
Epoch 86/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7430 - acc: 0.6177
Epoch 87/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7434 - acc: 0.6177
Epoch 88/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7416 - acc: 0.6185
Epoch 89/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7372 - acc: 0.6219
Epoch 90/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7461 - acc: 

18819/18819 [==============================] - 1s 59us/step - loss: 0.7269 - acc: 0.6200
Epoch 161/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7268 - acc: 0.6282
Epoch 162/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7253 - acc: 0.6306
Epoch 163/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7287 - acc: 0.6230
Epoch 164/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7244 - acc: 0.6261
Epoch 165/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7298 - acc: 0.6260: 0s - loss: 0
Epoch 166/500
18819/18819 [==============================] - 1s 58us/step - loss: 0.7274 - acc: 0.6279
Epoch 167/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7284 - acc: 0.6244
Epoch 168/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7260 - acc: 0.6282: 0s - loss: 0
Epoch 169/500
18819/18819 [==============================] 

18819/18819 [==============================] - 1s 64us/step - loss: 0.7234 - acc: 0.6306: 1s - loss:
Epoch 240/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7207 - acc: 0.6313
Epoch 241/500
18819/18819 [==============================] - 1s 65us/step - loss: 0.7221 - acc: 0.6290
Epoch 242/500
18819/18819 [==============================] - 1s 62us/step - loss: 0.7206 - acc: 0.6301
Epoch 243/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7184 - acc: 0.6332
Epoch 244/500
18819/18819 [==============================] - 1s 58us/step - loss: 0.7198 - acc: 0.6336
Epoch 245/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7167 - acc: 0.6270
Epoch 246/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7237 - acc: 0.6266
Epoch 247/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7191 - acc: 0.6292
Epoch 248/500
18819/18819 [==============================] - 1s 69us/step -

18819/18819 [==============================] - 1s 64us/step - loss: 0.7108 - acc: 0.6350
Epoch 319/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7127 - acc: 0.6323
Epoch 320/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7093 - acc: 0.6321
Epoch 321/500
18819/18819 [==============================] - 1s 63us/step - loss: 0.7119 - acc: 0.6327
Epoch 322/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7116 - acc: 0.6363
Epoch 323/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7138 - acc: 0.6352
Epoch 324/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7112 - acc: 0.6289
Epoch 325/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7130 - acc: 0.6339
Epoch 326/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7130 - acc: 0.6340
Epoch 327/500
18819/18819 [==============================] - 1s 63us/step - loss: 0.716

18819/18819 [==============================] - 1s 68us/step - loss: 0.7051 - acc: 0.6364
Epoch 398/500
18819/18819 [==============================] - 1s 73us/step - loss: 0.7081 - acc: 0.6377
Epoch 399/500
18819/18819 [==============================] - 1s 77us/step - loss: 0.7045 - acc: 0.6347
Epoch 400/500
18819/18819 [==============================] - 1s 75us/step - loss: 0.7065 - acc: 0.6391
Epoch 401/500
18819/18819 [==============================] - 1s 71us/step - loss: 0.7088 - acc: 0.6366
Epoch 402/500
18819/18819 [==============================] - 1s 70us/step - loss: 0.7085 - acc: 0.6337
Epoch 403/500
18819/18819 [==============================] - 1s 63us/step - loss: 0.7113 - acc: 0.6357
Epoch 404/500
18819/18819 [==============================] - 1s 64us/step - loss: 0.7057 - acc: 0.6380
Epoch 405/500
18819/18819 [==============================] - 1s 65us/step - loss: 0.7062 - acc: 0.6338
Epoch 406/500
18819/18819 [==============================] - 1s 62us/step - loss: 0.704

18819/18819 [==============================] - 1s 62us/step - loss: 0.7030 - acc: 0.6399
Epoch 477/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.6983 - acc: 0.6437
Epoch 478/500
18819/18819 [==============================] - 1s 63us/step - loss: 0.6999 - acc: 0.6442
Epoch 479/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.6995 - acc: 0.6450
Epoch 480/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.7000 - acc: 0.6414
Epoch 481/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.7039 - acc: 0.6399
Epoch 482/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7008 - acc: 0.6394
Epoch 483/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7005 - acc: 0.6437
Epoch 484/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.7004 - acc: 0.6392
Epoch 485/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.699

In [13]:
model.evaluate(x_test_snn, y_test_snn)

6274/6274 [==============================] - 0s 41us/step


[0.6848529712728988, 0.6405801721389863]

In [14]:
from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp, K.learning_phase()], outputs ) 

In [15]:
layer_outs_train = functor([x_train_snn, 1.])
layer_outs_test = functor([x_test_snn, 1.])

In [16]:
x_train_flat_snn = np.asarray(layer_outs_train[4]).reshape(-1,10)
x_test_flat_snn = np.asarray(layer_outs_test[4]).reshape(-1,10)

tsne = TSNE()
train_tsne_embeds_snn = tsne.fit_transform(x_train_flat_snn[:1000])
scatter(train_tsne_embeds_snn, y_train_snn[:1000], "Samples from Training Data LINCS (DenseNet) after")

eval_tsne_embeds_snn = tsne.fit_transform(x_test_flat_snn[:1000])
scatter(eval_tsne_embeds_snn, y_test_snn[:1000], "Samples from Validation Data LINCS (DenseNet) after")

/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
